In [ ]:
import tensorflow as tf

MODEL_PATH = "/content/drive/My Drive/leafnet_thesis/models/leafnet_v3.h5"

model = tf.keras.models.load_model(MODEL_PATH)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

with tf.io.gfile.GFile("/content/drive/My Drive/leafnet_thesis/models/leafnet_v3.tflite", "wb") as stream:
    stream.write(tflite_model)

In [ ]:
import os
from time import time

import numpy as np
import pandas as pd
import tensorflow as tf
from PIL import Image

DIR = "/content/drive/My Drive/leafnet_thesis/leaf_valid_v3"
CLASSES = sorted(os.listdir(DIR))
SIZE = (256, 256)

interpreter = tf.lite.Interpreter(model_path="/content/drive/My Drive/leafnet_thesis/models/leafnet_v3.tflite")
interpreter.allocate_tensors()

def predict_image(image_file):
    image = Image.open(image_file).convert("RGB").resize(SIZE, Image.ANTIALIAS)
    image = np.asarray(image)
    image = image.astype('float32')
    image /= 255.0
    tensor_index = interpreter.get_input_details()[0]['index']
    start = time()
    interpreter.set_tensor(tensor_index, [image])
    interpreter.invoke()
    duration = time() - start
    output_details = interpreter.get_output_details()[0]
    output = np.squeeze(interpreter.get_tensor(output_details['index']))
    if output_details['dtype'] == np.uint8:
        scale, zero_point = output_details['quantization']
        output = scale * (output - zero_point)
    return np.argmax(output), duration

def predict_class(class_directory, classes):
    classname = os.path.basename(class_directory)
    images = []
    for image in os.listdir(class_directory):
        images.append({"x": os.path.join(class_directory, image), "y": classname})
    correct = 0
    total_duration = 0
    for image in images:
        result, duration = predict_image(image["x"])
        if classes[result] == classname:
            correct += 1
        total_duration += duration
    score = correct / len(images)
    print(f"Score for {classname}: {score * 100:.2f} %.")
    print(f"Done in {total_duration:.3f} seconds for {len(images)} images.")
    return {"accuracy": score, "duration": total_duration}

def predict_classes(directory):
    results = {"class": [], "accuracy": [], "duration": []}
    classes = sorted(os.listdir(directory))
    total = 0
    for class_directory in classes:
        result = predict_class(os.path.join(directory, class_directory), classes)
        results["class"].append(class_directory)
        results["accuracy"].append(result["accuracy"])
        results["duration"].append(result["duration"])
        total += len(os.listdir(os.path.join(directory, class_directory)))
    print(pd.DataFrame(results))

predict_classes("/content/drive/My Drive/leafnet_thesis/leaf_valid_v3")
# predict_image("/content/drive/My Drive/leafnet_thesis/leaf_valid_v3/Apple___Apple_scab/1001.jpg")

NameError: ignored